# 11_Route Parameters

## 1. What is a Route Parameter?

* **Route parameters** are **placeholders** inside the URL (route) that let you capture **dynamic parts** of the URL.
* They make your API flexible and allow it to handle **different data based on the values in the URL**.
* Example idea:

  * `/users` → get **all** users (no dynamic part)
  * `/users/101` → get a **single** user whose **ID = 101** (dynamic part)
* In `/users/101`:

  * `/users` → **static** part (doesn’t change)
  * `101` → **dynamic** part (can be `1`, `20`, `1000`, etc.)
  * This dynamic segment is a **route parameter**.

---

## 2. Multiple Route Parameters in a URL

You can have **more than one** route parameter in a single URL.

Example:

```txt
/users/:gender/:city
```

* Possible URL: `/users/male/London`
* Here:

  * `gender` → dynamic (`male`, `female`, etc.)
  * `city` → dynamic (`London`, `Paris`, `Colombo`, etc.)
* Both `:gender` and `:city` are **route parameters**, and their **values can change**.

So:

* `/users/male/London`
* `/users/female/Paris`
* `/users/male/Colombo`

All match the **same route pattern**, just with **different parameter values**.

---

## 3. Defining a Route with Parameters in a Controller

Suppose you already have a controller method to get all users:

```ts
@Get('users')
getUsers() {
  // returns all users
}
```

Now you want a route to get a **single user by ID**:

### 3.1 Basic Route Parameter Syntax

```ts
@Get('users/:id')
getUserById() {
  // will handle requests like /users/1, /users/101, etc.
}
```

* `:id` is the **route parameter** name.
* Any request matching `/users/<some-id>` will hit this method.
* Example URLs:

  * `/users/1`
  * `/users/101`
  * `/users/2000`

### 3.2 Multiple Route Parameters in Decorator

You can define **multiple** parameters:

```ts
@Get('users/:id/:name/:gender')
getUserByDetails() {
  // will handle /users/1/John/male etc.
}
```

* Here, the method expects **3 parameters**: `id`, `name`, `gender`.
* All three are **mandatory** (for now).

If you call:

* `/users/1` → ❌ Not found (only 1 value)
* `/users/1/John` → ❌ Not found (2 values)
* `/users/1/John/male` → ✅ Matches the route pattern

---

## 4. Mandatory vs Optional Route Parameters

By default, **all route parameters are mandatory**.

```ts
@Get('users/:id/:name/:gender')
```

* Must supply all three values.

### 4.1 About "Optional" Route Parameters in Modern NestJS

Modern NestJS **does not support** optional route parameters using `?` directly in the path (like `:gender?`). Routes must have a concrete pattern.

If you want something to behave like an "optional" value, you can:

* Define **separate routes** for different patterns, or
* Use **query parameters** for values that may or may not be present.

#### 4.2 Using Separate Routes

```ts
// With gender
@Get('users/:id/:name/:gender')
getUserWithGender(
  @Param('id') id: string,
  @Param('name') name: string,
  @Param('gender') gender: string,
) {}

// Without gender
@Get('users/:id/:name')
getUserWithoutGender(
  @Param('id') id: string,
  @Param('name') name: string,
) {}
```

#### 4.3 Using Query Parameters for Optional Data

```ts
// Example URL: /users/1/John?gender=male
@Get('users/:id/:name')
getUser(
  @Param('id') id: string,
  @Param('name') name: string,
  @Query('gender') gender?: string, // optional
) {
  // gender may be undefined if not provided
}
```

> ✅ Summary: Instead of `:gender?` in the route path, use **separate routes** or **query parameters** to handle optional values.

## 5. Reading Route Parameter Values in a Controller

To **read** route parameter values, we use the `@Param()` decorator from `@nestjs/common`.

Make sure it is imported:

```ts
import { Controller, Get, Param } from '@nestjs/common';
```

### 5.1 Reading All Parameters as an Object

Example method:

```ts
@Get('users/:id/:name/:gender?')
getUserById(@Param() params: any) {
  console.log(params);
}
```

If we call: `/users/1/John/male`

Then `params` will be an object like:

```ts
{
  id: '1',
  name: 'John',
  gender: 'male'
}
```

Key points:

* `@Param()` **without arguments** returns **all route parameters** as an object.
* Keys = route parameter names (`id`, `name`, `gender`).
* Values = values from the URL.

### 5.2 Reading a Single Route Parameter

If you only care about **one** parameter (e.g. `id`), you can do this:

```ts
@Get('users/:id')
getUserById(@Param('id') id: string) {
  console.log(id); // e.g. '1'
}
```

* `@Param('id')` → reads *only* the `id` parameter.
* `id` will contain the value from URL (as a **string**).

You can do the same with other parameters:

```ts
@Get('users/:id/:name')
getUser(@Param('name') name: string) {
  console.log(name); // e.g. 'John'
}
```

### 5.3 Type of Route Parameters

> ⚠️ **Very important:**
> All route parameter values are read as **strings**, even if they look like numbers.

So if your URL is `/users/10`:

* `@Param('id') id` → `id` is `'10'` (string), **not** `10` (number).

If you need a **number** (for example to pass into a service method), you must **convert it manually**, e.g.:

```ts
const numericId = +id; // converts '10' → 10
```

Or use `Number(id)`.

---

## 6. Using Route Parameter in a Real Controller Method

### 6.1 Basic Controller Setup

Assume you have a `UserService` with a method:

```ts
getUserById(id: number) {
  // returns user with given id
}
```

In the controller:

```ts
@Controller('users')
export class UsersController {
  constructor(private readonly userService: UserService) {}

  @Get(':id')
  getUserById(@Param('id') id: string) {
    // id is a string from the URL

    const numericId = +id; // convert string to number

    // call service method with number
    const user = this.userService.getUserById(numericId);

    // return the user as the response
    return user;
  }
}
```

What’s happening here:

1. Client calls `/users/1`.
2. Nest matches this URL with `@Get(':id')`.
3. `@Param('id') id` captures `"1"` from the URL.
4. We convert `id` to a number using `+id`.
5. We pass `numericId` to `userService.getUserById(...)`.
6. We return the user object from the controller → sent as response.

### 6.2 Order of Route Parameter Values

If you have multiple parameters like:

```ts
@Get('users/:id/:name/:gender?')
```

Then for a URL like `/users/1/John/male`:

* `id` → `1`
* `name` → `John`
* `gender` → `male`

The **order in the URL** must match the **order in the route pattern**.

---